## 【1】最初にこのプログラムを実行してください。
その日最初に作業するときなど。（10～20秒かかります）

In [ ]:
!apt-get update
!apt install -y chromium-chromedriver fonts-ipafont fonts-ipaexfont
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install gspread

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from time import sleep
from google.colab import auth, drive
from oauth2client.client import GoogleCredentials
import gspread

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--window-size=1280,1400')
driver = webdriver.Chrome('chromedriver',options=options)

#Spreadsheetとの連携
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

ss_id = "1Wy8g4zwRA2tR1jFrfpHSJUnRwpvm4HYOHNomPUb7OEw"
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("東京")


#求人情報詳細ページ上での処理を関数定義
#sleep()の数字で秒数を調整してください。
def page():

    #「のコピー」がない案件名を入力
    shokushu_name = driver.find_element_by_xpath('//*[@id="job_offer_title"]')
    shokushu_name.clear()
    shokushu_name.send_keys(anken_list[anken_num])
    sleep(1)

    #郵便番号をSpreadsheetから引っ張ってくる
    yubin_bangou = driver.find_element_by_xpath('//*[@id="job_offer_work_post"]')
    yubin_bangou.clear()
    yubin_bangou.send_keys(postcode)
    sleep(2)

    #下書き→公開のプルダウン変更
    status = driver.find_element_by_xpath('//*[@id="job_offer_status"]')
    select = Select(status)
    select.select_by_index(1)
    sleep(1)

    #保存ボタン押下
    hozon = driver.find_element_by_id('save_button')
    hozon.click()
    sleep(2)

#案件名
anken_list = ['基本給26万円以上の事務','基本給26万円以上のコールセンター','未経験歓迎事務','事務・入社祝金10万円','事務・コールセンター業']

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

## 【2】こちらが、サイトへの記入作業を実行するプログラムです。

※Googleの仕様で、一度にSpread Sheetにアクセスできる回数が1000件までと決まっているようです。郵便番号が1000件以上ある場合は、複数回に分けて実施してください。2回以上続けて作業する場合は、【1】は飛ばして【2】のみを実行するだけで大丈夫です。

In [ ]:
#ログイン処理
url = 'https://m.q-mate.jp/user_sessions/new?layout=login'
driver.get(url)
#sleep(1)
elem_username = driver.find_element_by_id('login_id')
elem_username.send_keys('topjob.kyujin@gmail.com')
elem_password = driver.find_element_by_id('password')
elem_password.send_keys('topjob9283719')
#sleep(1)
elem_login_btn = driver.find_element_by_id('login_button')
elem_login_btn.click()
#sleep(1)
url_2 = 'https://m.q-mate.jp/job_offers?direction=desc&sort=updated_at'
driver.get(url_2)
sleep(2)


#求人情報詳細ページ上での入力作業
for i in range(200):

    fukusei = driver.find_element_by_xpath('//*[@id="list_table"]/tbody/tr[1]/td[7]/div/div[1]/a[2]')
    fukusei.click()
   
    #郵便番号を1つ引用
    postcode = worksheet.acell(f'A{i+1}').value
    
    #5種類の案件を順繰りに入力していく処理
    if int(str(i)[-1]) == 0 or int(str(i)[-1]) == 5:
        anken_num = 0
        page()

    elif int(str(i)[-1]) == 1 or int(str(i)[-1]) == 6:
        anken_num = 1
        page()

    elif int(str(i)[-1]) == 2 or int(str(i)[-1]) == 7:
        anken_num = 2
        page()
        
    elif int(str(i)[-1]) == 3 or int(str(i)[-1]) == 8:
        anken_num = 3
        page()
        
    else:
        anken_num = 4
        page()
    
    driver.get(url_2)
    sleep(1)

else:
    print('=============================')
    print('すべての処理が完了しました。')
    print('=============================')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning

すべての処理が完了しました。


In [ ]:
#driver.save_screenshot('result.png')